# WRDS Feature Extraction: 20 Core Features

**In the code, feature names will be renamed to project standard names. The column names used in the actual data file are as follows:**

## Feature List (Actual Column Names)

### Momentum (5 )
- **ret_1_0** (orig: mom1m) - Past 1 month return
- **ret_6_1** (orig: mom6m) - Past 6 months return (skip most recent 1 month)
- **ret_12_1** (orig: mom12m) - Past 12 months return (skip most recent 1 month)
- **ret_36_1** (orig: mom36m) - Past 36 months return
- **chmom** - Momentum change = ret_12_1 - ret_1_0

### Reversal/Price Behavior (2 )
- **rmax1_21d** (orig: maxret) - Maximum return over past 21 days
- **rvol_21d** (orig: retvol) - Return volatility over past 21 days

### Liquidity (5 )
- **turnover_126d** (orig: turn) - Average turnover over past 126 days
- **std_turn** - Turnover standard deviation
- **dolvol_126d** (orig: dolvol) - Dollar volume over past 126 days
- **bidaskhl_21d** (orig: baspread) - Bid-ask spread
- **zero_trades_252d** (orig: zerotrade) - Zero trade days ratio over past 252 days

### Size/Valuation (3 )
- **me** (orig: mve) - Market capitalization
- **be_me** (orig: bm) - Book-to-market ratio
- **cashpr** - Cash price ratio

### Profitability (2 )
- **qmj_prof** (orig: operprof) - Profitability
- **roeq** - Return on equity

### Risk (3 )
- **beta_60m** (orig: beta) - 60-month Beta
- **betasq** - Beta squared
- **ivol_capm_252d** (orig: idiovol) - CAPM residual volatility

### Industry (1 feature)
- **sic2** - 2-digit SIC industry code

**Total: 20 core **

## 📋 Feature Name Mapping Table

| Paper/Original Name | Project Variable Name | Description |
|------------|----------|------|
| mom1m | ret_1_0 | Past 1 month return |
| mom6m | ret_6_1 | Past 6 months return (skip most recent 1 month) |
| mom12m | ret_12_1 | Past 12 months return (skip most recent 1 month) |
| mom36m | ret_36_1 | Past 36 months return |
| maxret | rmax1_21d | Maximum return over past 21 days |
| retvol | rvol_21d | Return volatility over past 21 days |
| turn | turnover_126d | Average turnover over past 126 days |
| dolvol | dolvol_126d | Dollar volume over past 126 days |
| baspread | bidaskhl_21d | Bid-ask spread |
| zerotrade | zero_trades_252d | Zero trade days ratio over past 252 days |
| mve | me | Market capitalization |
| bm | be_me | Book-to-market ratio |
| beta | beta_60m | 60-month Beta |
| idiovol | ivol_capm_252d | CAPM residual volatility |
| operprof | qmj_prof | Profitability |


## 1. Import Libraries and Configure Parameters


In [1]:
import wrds
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# Configuration Parameters
# ============================================================================
START_DATE = '2018-01-01'  # Data start date
END_DATE = '2024-12-31'    # Data end date
OUTPUT_FILE = 'wrds_20_.csv'  # Output filename

print("✅ Libraries imported successfully！")
print(f"📅 Data date range: {START_DATE} to {END_DATE}")


✅ Libraries imported successfully！
📅 Data date range: 2018-01-01 to 2024-12-31


## 2. Connect to WRDS Database


In [ ]:
print("Connecting to WRDS...")
db = wrds.Connection()
print("✅ WRDS connection successful！")


### Optional: Check CRSP Daily table structure

If you need to see what columns are in the table, you can run the code below:


In [ ]:
# Optional: Check CRSP Daily table column names
# Uncomment the code below to view table structure
columns_info = db.raw_sql("""
     SELECT column_name, data_type
     FROM information_schema.columns
     WHERE table_schema = 'crsp' 
       AND table_name = 'dsf'
     ORDER BY ordinal_position
     LIMIT 50
 """)
print("First 50 columns of CRSP Daily (dsf) table:")
print(columns_info)


First 50 columns of CRSP Daily (dsf) table:
   column_name          data_type
0        cusip  character varying
1       permno            integer
2       permco            integer
3       issuno            integer
4        hexcd           smallint
5       hsiccd            integer
6         date               date
7        bidlo            numeric
8        askhi            numeric
9          prc            numeric
10         vol            numeric
11         ret            numeric
12         bid            numeric
13         ask            numeric
14      shrout   double precision
15      cfacpr   double precision
16     cfacshr   double precision
17     openprc            numeric
18      numtrd            integer
19        retx            numeric


## 3. Extract Momentum  (5 )

Calculate the following momentum indicators:
- **mom1m**: Past 1 month return
- **mom6m**: Past 6 months return (skip most recent 1 month)
- **mom12m**: Past 12 months return (skip most recent 1 month)
- **mom36m**: Past 36 months return (skip most recent 1 month)
- **chmom**: Momentum change = mom12m - mom1m


In [ ]:
print("📊 Step 1: Extracting Momentum ...")

# Get daily returns from CRSP Daily
# Include bid and ask columns for more accurate Bid-ask spread calculation, and hsiccd for Industry classification
print("   - Getting daily return data from CRSP Daily...")
try:
    crsp_daily = db.raw_sql(f"""
        SELECT permno, date, ret, vol, prc, shrout, askhi, bidlo, bid, ask, hsiccd
        FROM crsp.dsf
        WHERE date >= '{START_DATE}' 
          AND date <= '{END_DATE}'
          AND ret IS NOT NULL
        ORDER BY permno, date
    """)
    print("   ✅ SQL query successful")
except Exception as e:
    print(f"   ⚠️  Query with hsiccd failed: {str(e)[:200]}")
    print("   - Trying query without hsiccd...")
    crsp_daily = db.raw_sql(f"""
        SELECT permno, date, ret, vol, prc, shrout, askhi, bidlo, bid, ask
        FROM crsp.dsf
        WHERE date >= '{START_DATE}' 
          AND date <= '{END_DATE}'
          AND ret IS NOT NULL
        ORDER BY permno, date
    """)

crsp_daily['date'] = pd.to_datetime(crsp_daily['date'])
crsp_daily = crsp_daily.sort_values(['permno', 'date'])

print(f"   ✅ Retrieved {len(crsp_daily):,} daily data records")
print(f"   ✅ Number of stocks: {crsp_daily['permno'].nunique():,}")
print(f"   ✅ Data columns: {list(crsp_daily.columns)}")
if 'hsiccd' in crsp_daily.columns:
    print(f"   ✅ hsiccd column exists, valid values: {crsp_daily['hsiccd'].notna().sum():,}")
else:
    print(f"   ⚠️  hsiccd column does not exist")
crsp_daily.head()


📊 Step 1: Extracting momentum ...
   - Getting daily return data from CRSP Daily...
   ✅ Retrieved 15,008,295 daily data records
   ✅ Number of stocks: 13,854


,permno,date,ret,vol,prc,shrout,askhi,bidlo,bid,ask
0,10026,2018-01-02,-0.017454,190618.0,149.17999,18668.0,152.39,147.85001,149.17999,149.42
1,10026,2018-01-03,-0.009988,63693.0,147.69,18668.0,150.27,146.68359,147.41,147.69
2,10026,2018-01-04,0.013813,127552.0,149.73,18668.0,149.99001,147.77,149.45,149.72
3,10026,2018-01-05,-0.00955,44647.0,148.3,18668.0,150.89,148.25999,148.28999,148.39
4,10026,2018-01-08,0.000742,55014.0,148.41,18668.0,150.96899,146.20821,148.21001,148.34


In [ ]:
# Calculating momentum 
print("   - Calculating momentum ...")

def calc_momentum(group):
    """Calculate various momentum indicators"""
    ret = group['ret'].values
    
    # Convert to numpy array for fast calculation
    ret_plus_one = 1 + ret
    ret_plus_one[np.isnan(ret_plus_one)] = 1
    
    # mom1m: Past 1  months (21  trading days)
    mom1m = np.full(len(group), np.nan)
    for i in range(21, len(group)):
        mom1m[i] = np.prod(ret_plus_one[i-21:i]) - 1
    
    # mom6m: Past 6 months, skip most recent 1 month (21-168 trading days)
    mom6m = np.full(len(group), np.nan)
    for i in range(168, len(group)):
        mom6m[i] = np.prod(ret_plus_one[i-168:i-21]) - 1
    
    # mom12m: Past 12 months, skip most recent 1 month (21-252 trading days)
    mom12m = np.full(len(group), np.nan)
    for i in range(252, len(group)):
        mom12m[i] = np.prod(ret_plus_one[i-252:i-21]) - 1
    
    # mom36m: Past 36 months, skip most recent 1 month (21-756 trading days)
    mom36m = np.full(len(group), np.nan)
    for i in range(756, len(group)):
        mom36m[i] = np.prod(ret_plus_one[i-756:i-21]) - 1
    
    return pd.DataFrame({
        'mom1m': mom1m,
        'mom6m': mom6m,
        'mom12m': mom12m,
        'mom36m': mom36m
    }, index=group.index)

momentum_ = crsp_daily.groupby('permno', group_keys=False).apply(calc_momentum)
crsp_daily = pd.concat([crsp_daily, momentum_], axis=1)

# chmom: Momentum change
crsp_daily['chmom'] = crsp_daily['mom12m'] - crsp_daily['mom1m']

print("   ✅ Momentum  calculated")
print(f"   - mom1m: {crsp_daily['mom1m'].notna().sum():,}  valid values")
print(f"   - mom12m: {crsp_daily['mom12m'].notna().sum():,}  valid values")


   - Calculating momentum ...
   ✅ Momentum  calculated
   - mom1m: 14,718,770  valid values
   - mom12m: 11,739,553  valid values


## 4. Extract Reversal/Price Behavior  (2 )

- **maxret**: Maximum daily return over past 21 days
- **retvol**: Return volatility over past 21 days


In [ ]:
print("📊 Step 2: Extracting reversal/price behavior ...")

# maxret: Maximum daily return over past 21 days
crsp_daily['maxret'] = crsp_daily.groupby('permno')['ret'].transform(
    lambda x: x.rolling(21, min_periods=10).max()
)

# retvol: Return volatility over past 21 days
crsp_daily['retvol'] = crsp_daily.groupby('permno')['ret'].transform(
    lambda x: x.rolling(21, min_periods=10).std()
)

print("   ✅ Reversal  calculated")


📊 Step 2: Extracting reversal/price behavior ...
   ✅ Reversal  calculated


## 5. Extract Liquidity  (5 )

- **turn**: Turnover = volume / shares outstanding
- **std_turn**: Turnover volatility
- **dolvol**: Dollar volume = volume × price
- **zerotrade**: Zero trade days ratio (past 252 days)
- **baspread**: Bid-ask spread（Prefer using bid/ask, otherwise use askhi/bidlo）


In [ ]:
print("📊 Step 3: Extracting liquidity ...")

# turn: Turnover = volume / shares outstanding
crsp_daily['turn'] = crsp_daily['vol'] / crsp_daily['shrout']
crsp_daily['turn_126d'] = crsp_daily.groupby('permno')['turn'].transform(
    lambda x: x.rolling(126, min_periods=60).mean()
)

# std_turn: Turnover volatility
crsp_daily['std_turn'] = crsp_daily.groupby('permno')['turn'].transform(
    lambda x: x.rolling(126, min_periods=60).std()
)

# dolvol: Dollar volume = volume * price
crsp_daily['dolvol'] = crsp_daily['vol'] * crsp_daily['prc']
crsp_daily['dolvol_126d'] = crsp_daily.groupby('permno')['dolvol'].transform(
    lambda x: x.rolling(126, min_periods=60).mean()
)

# zerotrade: Zero trade days ratio (past 252 days)
# Handle missing values: missing vol is treated as non-zero (i.e., not a zero trade day), only vol==0 is marked as 1
crsp_daily['is_zero'] = ((crsp_daily['vol'] == 0) & (crsp_daily['vol'].notna())).astype(int)
crsp_daily['zerotrade'] = crsp_daily.groupby('permno')['is_zero'].transform(
    lambda x: x.rolling(252, min_periods=120).mean()
)

# baspread: Bid-ask spread
# Prefer using bid and ask for calculation (more accurate), otherwise use askhi and bidlo
if 'bid' in crsp_daily.columns and 'ask' in crsp_daily.columns:
    # Calculate Bid-ask spread using bid and ask:(ask - bid) / mid_price
    crsp_daily['mid_price'] = (crsp_daily['bid'] + crsp_daily['ask']) / 2
    crsp_daily['baspread_est'] = (crsp_daily['ask'] - crsp_daily['bid']) / crsp_daily['mid_price']
    # Handle missing values: if bid/ask is missing, use askhi/bidlo
    mask = crsp_daily['baspread_est'].isna()
    if mask.any() and 'askhi' in crsp_daily.columns and 'bidlo' in crsp_daily.columns:
        crsp_daily.loc[mask, 'baspread_est'] = (
            (crsp_daily.loc[mask, 'askhi'] - crsp_daily.loc[mask, 'bidlo']) / 
            crsp_daily.loc[mask, 'prc']
        )
elif 'askhi' in crsp_daily.columns and 'bidlo' in crsp_daily.columns:
    # Calculate using askhi and bidlo
    crsp_daily['baspread_est'] = (crsp_daily['askhi'] - crsp_daily['bidlo']) / crsp_daily['prc']
else:
    # If neither is available, use return volatility as a proxy
    print("   ⚠️  bid/ask or askhi/bidlo columns not found, using return volatility as liquidity proxy")
    crsp_daily['baspread_est'] = crsp_daily.groupby('permno')['ret'].transform(
        lambda x: x.rolling(21, min_periods=10).std()
    )

# Calculate 21-day average Bid-ask spread
crsp_daily['baspread_21d'] = crsp_daily.groupby('permno')['baspread_est'].transform(
    lambda x: x.rolling(21, min_periods=10).mean()
)

print("   ✅ Liquidity  calculated")


📊 Step 3: Extracting liquidity ...
   ✅ Liquidity  calculated


## 6. Extract Size/Valuation  (3 )

- **mve (size)**: Market capitalization = price × shares outstanding
- **bm**: Book-to-market ratio = book value / Market capitalization
- **cashpr**: cash price ratio


In [ ]:
print("📊 Step 4: Extracting size/valuation ...")

# mve: Market capitalization（more accurate from CRSP Monthly）
print("   - Getting Market capitalization data from CRSP Monthly...")
crsp_monthly = db.raw_sql(f"""
    SELECT permno, date, prc, shrout
    FROM crsp.msf
    WHERE date >= '{START_DATE}' 
      AND date <= '{END_DATE}'
    ORDER BY permno, date
""")

crsp_monthly['date'] = pd.to_datetime(crsp_monthly['date'])
crsp_monthly['mve'] = crsp_monthly['prc'] * crsp_monthly['shrout']  # In millions of USD

print(f"   ✅ Retrieved {len(crsp_monthly):,} monthly data records")
crsp_monthly.head()


📊 Step 4: Extracting size/valuation ...
   - Getting market capitalization data from CRSP Monthly...
   ✅ Retrieved 729,493 monthly data records


,permno,date,prc,shrout,mve
0,10026,2018-01-31,138.44,18678.0,2585782.32
1,10026,2018-02-28,134.33,18678.0,2509015.74
2,10026,2018-03-29,136.56,18697.0,2553262.32
3,10026,2018-04-30,137.41,18702.0,2569841.82
4,10026,2018-05-31,141.62,18702.0,2648577.24


In [ ]:
# bm: Book-to-market ratio（requires Compustat）
print(" - Getting book value data from Compustat...")
compustat = db.raw_sql(f"""
    SELECT gvkey, datadate, ceq, at, che, act, lt
    FROM comp.funda
    WHERE datadate >= '{START_DATE}' 
      AND datadate <= '{END_DATE}'
      AND indfmt = 'INDL'
      AND datafmt = 'STD'
      AND popsrc = 'D'
      AND consol = 'C'
    ORDER BY gvkey, datadate
""")

compustat['datadate'] = pd.to_datetime(compustat['datadate'])

# Link Compustat and CRSP
print("   - Link Compustat and CRSP...")
link_table = db.raw_sql("""
    SELECT gvkey, lpermno as permno, linkdt, linkenddt
    FROM crsp.ccmxpf_linktable
    WHERE linktype IN ('LU', 'LC')
""")

link_table['linkdt'] = pd.to_datetime(link_table['linkdt'])
link_table['linkenddt'] = pd.to_datetime(link_table['linkenddt'])

# Merge Compustat and CRSP
compustat['year'] = compustat['datadate'].dt.year
compustat['month'] = compustat['datadate'].dt.month

# Merge link table
compustat_linked = compustat.merge(
    link_table, on='gvkey', how='inner'
).query('datadate >= linkdt & (linkenddt.isna() | datadate <= linkenddt)')

# Only keep December data for matching (fiscal year end is usually in December)
compustat_dec = compustat_linked[compustat_linked['month'] == 12].copy()
compustat_dec = compustat_dec[['permno', 'year', 'ceq']].rename(columns={'ceq': 'be'})

# Merge to CRSP monthly data
crsp_monthly['year'] = crsp_monthly['date'].dt.year
crsp_monthly['month'] = crsp_monthly['date'].dt.month

crsp_monthly = crsp_monthly.merge(
    compustat_dec, 
    left_on=['permno', 'year'], 
    right_on=['permno', 'year'], 
    how='left'
)
crsp_monthly['be'] = crsp_monthly.groupby('permno')['be'].ffill()

# Calculate be_me
crsp_monthly['bm'] = crsp_monthly['be'] / (crsp_monthly['mve'] * 1e6)  # be is in millions, mve is also in millions

print(" ✅ Book-to-market ratio calculated")


   - Getting book value data from Compustat...
   - Link Compustat and CRSP...
   ✅ Book-to-market ratio calculated


In [ ]:
# cashpr: cash price ratio（requires Compustat data）
# First filter December data, then select needed columns
compustat_cash = compustat_linked[compustat_linked['month'] == 12][['permno', 'year', 'che', 'act', 'lt']].copy()
compustat_cash = compustat_cash.merge(
    crsp_monthly[crsp_monthly['month'] == 12][['permno', 'year', 'mve']],
    on=['permno', 'year'],
    how='left'
)

compustat_cash['cashpr'] = (
    compustat_cash['che'] + 0.75 * compustat_cash['act'] - 0.5 * compustat_cash['lt']
) / (compustat_cash['mve'] * 1e6)

print(" ✅ Cash price ratio calculated")
print(" ✅ Size/valuation  calculated")


   ✅ Cash price ratio calculated
   ✅ Size/valuation  calculated


## 7. Extract Profitability  (2 )

- **operprof**: Operating profit margin = (revenue - cost of goods sold - selling, general and administrative expenses - interest expense) / shareholders equity
- **roeq**: Return on equity = net income / shareholders equity


In [ ]:
print("📊 Step 5: Extracting Profitability ...")

compustat_prof = db.raw_sql(f"""
    SELECT gvkey, datadate, revt, cogs, xsga, xint, ceq, ib
    FROM comp.funda
    WHERE datadate >= '{START_DATE}' 
      AND datadate <= '{END_DATE}'
      AND indfmt = 'INDL'
      AND datafmt = 'STD'
      AND popsrc = 'D'
      AND consol = 'C'
    ORDER BY gvkey, datadate
""")

compustat_prof['datadate'] = pd.to_datetime(compustat_prof['datadate'])
compustat_prof['year'] = compustat_prof['datadate'].dt.year
compustat_prof['month'] = compustat_prof['datadate'].dt.month

# Link to CRSP
compustat_prof_linked = compustat_prof.merge(
    link_table, on='gvkey', how='inner'
).query('datadate >= linkdt & (linkenddt.isna() | datadate <= linkenddt)')

# operprof: Operating profit margin
compustat_prof_linked['operprof'] = (
    compustat_prof_linked['revt'] - compustat_prof_linked['cogs'] 
    - compustat_prof_linked['xsga'] - compustat_prof_linked['xint']
) / compustat_prof_linked['ceq']

# roeq: Return on equity
compustat_prof_linked['roeq'] = compustat_prof_linked['ib'] / compustat_prof_linked['ceq']

# Only keep year-end data
compustat_prof_dec = compustat_prof_linked[
    compustat_prof_linked['month'] == 12
][['permno', 'year', 'operprof', 'roeq']]

print(" ✅ Profitability  calculated")
print(f" - Valid operprof values: {compustat_prof_dec['operprof'].notna().sum():,}")
print(f" - Valid roeq values: {compustat_prof_dec['roeq'].notna().sum():,}")


📊 Step 5: Extracting Profitability ...
   ✅ Profitability  calculated
   - Valid operprof values: 20,715
   - Valid roeq values: 29,763


## 8. Extract Risk  (3 )

- **beta**: CAPM beta（using past 252 trading days to regress on market returns）
- **betasq**: beta squared
- **idiovol**: idiosyncratic volatility (standard deviation of CAPM residuals)


In [ ]:
print("📊 Step 6: Extracting risk ...")

# Get market returns
print("   - Getting market return data...")
market_ret = db.raw_sql(f"""
    SELECT date, vwretd as mkt_ret
    FROM crsp.dsi
    WHERE date >= '{START_DATE}' 
      AND date <= '{END_DATE}'
""")

market_ret['date'] = pd.to_datetime(market_ret['date'])

# Merge stock and market returns
crsp_beta = crsp_daily[['permno', 'date', 'ret']].merge(
    market_ret, on='date', how='left'
)

print(f"   ✅ Retrieved {len(market_ret):,} market return data records")


📊 Step 6: Extracting risk ...
   - Getting market return data...
   ✅ Retrieved 1,761 market return data records


In [ ]:
# Calculate beta (simplified version: using past 252  trading days)
print("   - Calculating CAPM beta...")

def calc_beta_simple(group):
    """Simplified beta calculation"""
    ret = group['ret'].values
    mkt_ret = group['mkt_ret'].values
    
    beta = np.full(len(group), np.nan)
    for i in range(252, len(group)):
        y = ret[i-252:i]
        x = mkt_ret[i-252:i]
        mask = ~(np.isnan(y) | np.isnan(x))
        if mask.sum() >= 60:
            cov = np.cov(y[mask], x[mask])[0,1]
            var = np.var(x[mask])
            if var > 0:
                beta[i] = cov / var
    
    return pd.Series(beta, index=group.index)

crsp_beta['beta'] = crsp_beta.groupby('permno', group_keys=False).apply(calc_beta_simple).values
crsp_beta['betasq'] = crsp_beta['beta'] ** 2

print("   ✅ Beta calculated")


   - Calculating CAPM beta...
   ✅ Beta calculated


In [ ]:
# Calculate idiovol (idiosyncratic volatility)
print("   - Calculating idiosyncratic volatility...")

def calc_idiovol_simple(group):
    """Simplified idiovol calculation"""
    ret = group['ret'].values
    mkt_ret = group['mkt_ret'].values
    
    idiovol = np.full(len(group), np.nan)
    for i in range(252, len(group)):
        y = ret[i-252:i]
        x = mkt_ret[i-252:i]
        mask = ~(np.isnan(y) | np.isnan(x))
        if mask.sum() >= 60:
            # Simple linear regression
            x_clean = x[mask]
            y_clean = y[mask]
            beta = np.cov(y_clean, x_clean)[0,1] / np.var(x_clean)
            alpha = np.mean(y_clean) - beta * np.mean(x_clean)
            residuals = y_clean - (alpha + beta * x_clean)
            idiovol[i] = np.std(residuals)
    
    return pd.Series(idiovol, index=group.index)

crsp_beta['idiovol'] = crsp_beta.groupby('permno', group_keys=False).apply(calc_idiovol_simple).values

print("   ✅ Idiosyncratic volatility calculated")
print("   ✅ Risk  calculated")


   - Calculating idiosyncratic volatility...
   ✅ Idiosyncratic volatility calculated
   ✅ Risk  calculated


## 9. Extract Industry variable (1 )

- **sic2**: First 2 digits of SIC industry code


In [ ]:
print("📊 Step 7: Extracting Industry ...")

# Option 1: Prefer getting hsiccd from crsp.dsf (if available, faster and no additional query needed)
print("   - Check if hsiccd can be extracted from crsp_daily...")
print(f"   - Columns of crsp_daily: {list(crsp_daily.columns)}")

if 'hsiccd' in crsp_daily.columns:
    print("   ✅ Found hsiccd column, extracting directly from crsp_daily...")
    crsp_names = crsp_daily[['permno', 'date', 'hsiccd']].copy()
    crsp_names = crsp_names.rename(columns={'hsiccd': 'siccd'})
    crsp_names = crsp_names.dropna(subset=['siccd'])
    print(f"   - Extracted {len(crsp_names):,}  records")
else:
    print("   ⚠️  hsiccd column not found in crsp_daily, trying to query from crsp.stocknames...")
    # Option 2: Query from crsp.stocknames (requires rollback handling)
    # Note: crsp.stocknames uses namedt and nameenddt to represent date range, not a single date column
    unique_permnos = crsp_daily['permno'].unique()
    print(f"   - Need to query {len(unique_permnos):,}   stocks")
    print("   - Note: crsp.stocknames uses date range columns (namedt, nameenddt)")
    
    # First try rollback to clean previous error transactions
    try:
        db.connection.rollback()
        print("   - Database connection cleaned")
    except:
        pass
    
    # Optimized method: First get all stocknames data, then merge with crsp_daily dates
    print("   - Method: Get stocknames data, then merge with crsp_daily dates...")
    
    # Split permno list into multiple batches
    batch_size = 1000  # Increase batch size to reduce number of queries
    stocknames_list = []
    
    for i in range(0, len(unique_permnos), batch_size):
        batch_permnos = unique_permnos[i:i+batch_size]
        permno_str = ','.join(map(str, batch_permnos))
        
        try:
            db.connection.rollback()
            batch_data = db.raw_sql(f"""
                SELECT permno, namedt, nameenddt, siccd
                FROM crsp.stocknames
                WHERE permno IN ({permno_str})
                  AND (nameenddt IS NULL OR nameenddt >= '{START_DATE}')
                  AND namedt <= '{END_DATE}'
            """)
            if len(batch_data) > 0:
                stocknames_list.append(batch_data)
            print(f"   - Processed {min(i+batch_size, len(unique_permnos))}/{len(unique_permnos)}   stocks")
        except Exception as e:
            print(f"   ⚠️  Batch {i//batch_size + 1} Query failed: {str(e)[:100]}")
            try:
                db.connection.rollback()
            except:
                pass
            continue
    
    if stocknames_list:
        # Merge all batches
        stocknames_all = pd.concat(stocknames_list, ignore_index=True)
        stocknames_all['namedt'] = pd.to_datetime(stocknames_all['namedt'])
        stocknames_all['nameenddt'] = pd.to_datetime(stocknames_all['nameenddt'])
        
        # Merge with crsp_daily dates: For each permno-date, find corresponding siccd
        print("   - Merge with crsp_daily dates (using vectorized method)...")
        crsp_daily_dates = crsp_daily[['permno', 'date']].copy()
        
        # Use merge_asof for efficient matching (group by permno, match by nearest date)
        # First prepare stocknames data for each permno
        crsp_names_list = []
        for permno in crsp_daily_dates['permno'].unique():
            permno_dates = crsp_daily_dates[crsp_daily_dates['permno'] == permno][['date']].copy()
            permno_stocknames = stocknames_all[stocknames_all['permno'] == permno].copy()
            
            if len(permno_stocknames) == 0:
                continue
            
            # For each date, find valid siccd
            siccd_list = []
            for date in permno_dates['date']:
                valid = permno_stocknames[
                    (permno_stocknames['namedt'] <= date) &
                    ((permno_stocknames['nameenddt'].isna()) | (permno_stocknames['nameenddt'] >= date))
                ]
                if len(valid) > 0:
                    # Take the one with largest namedt (most recent)
                    siccd_list.append(valid.loc[valid['namedt'].idxmax(), 'siccd'])
                else:
                    siccd_list.append(None)
            
            permno_dates['permno'] = permno
            permno_dates['siccd'] = siccd_list
            crsp_names_list.append(permno_dates)
        
        if crsp_names_list:
            crsp_names = pd.concat(crsp_names_list, ignore_index=True)
            crsp_names = crsp_names[['permno', 'date', 'siccd']].dropna(subset=['siccd'])
            print(f"   - Successfully matched {len(crsp_names):,}  records")
        else:
            crsp_names = pd.DataFrame(columns=['permno', 'date', 'siccd'])
            print("   ⚠️  Failed to match any records")
    else:
        print("   ⚠️  Unable to get data from stocknames, creating empty data frame")
        crsp_names = pd.DataFrame(columns=['permno', 'date', 'siccd'])

# Process data
if len(crsp_names) > 0:
    crsp_names['date'] = pd.to_datetime(crsp_names['date'])
    crsp_names['sic2'] = crsp_names['siccd'].astype(str).str[:2]
    print("   ✅ Industry features extracted")
    print(f"   - Unique industry code count: {crsp_names['sic2'].nunique()}")
else:
    print("   ⚠️  Failed to get Industry data, creating empty data frame")
    crsp_names = pd.DataFrame(columns=['permno', 'date', 'siccd', 'sic2'])


📊 Step 7: Extracting industry ...
   - Check if hsiccd can be extracted from crsp_daily...
   - Columns of crsp_daily: ['permno', 'date', 'ret', 'vol', 'prc', 'shrout', 'askhi', 'bidlo', 'bid', 'ask', 'mom1m', 'mom6m', 'mom12m', 'mom36m', 'chmom', 'maxret', 'retvol', 'turn', 'turn_126d', 'std_turn', 'dolvol', 'dolvol_126d', 'is_zero', 'zerotrade', 'mid_price', 'baspread_est', 'baspread_21d']
   ⚠️  hsiccd column not found in crsp_daily, trying to query from crsp.stocknames...
   - Need to query 13,854  stocks
   - Note: crsp.stocknames uses date range columns (namedt, nameenddt)
   - Database connection cleaned
   - Method: Get stocknames data, then merge with crsp_daily dates...
   - Processed 1000/13854  stocks
   - Processed 2000/13854  stocks
   - Processed 3000/13854  stocks
   - Processed 4000/13854  stocks
   - Processed 5000/13854  stocks
   - Processed 6000/13854  stocks
   - Processed 7000/13854  stocks
   - Processed 8000/13854  stocks
   - Processed 9000/13854  stocks
   - 

## 10. Merge all features to monthly frequency

Aggregate all daily features to monthly frequency (take end-of-month values), and merge all features into one data frame.


In [ ]:
print("📊 Step 8: Merge all features to monthly frequency...")

# Aggregate daily data to monthly (take end-of-month values)
crsp_daily['year'] = crsp_daily['date'].dt.year
crsp_daily['month'] = crsp_daily['date'].dt.month

# Select end-of-month data
daily_monthly = crsp_daily.groupby(['permno', 'year', 'month']).last().reset_index()
daily_monthly['date'] = pd.to_datetime(daily_monthly[['year', 'month']].assign(day=1))

# Merge all features (note: need to keep year and month columns for subsequent merge)
_monthly = daily_monthly[[
    'permno', 'date', 'year', 'month', 'mom1m', 'mom6m', 'mom12m', 'mom36m', 'chmom',
    'maxret', 'retvol', 'turn_126d', 'std_turn', 'dolvol_126d', 
    'zerotrade', 'baspread_21d'
]].copy()

print(f"   ✅ Daily data aggregation completed, total {len(_monthly):,}  monthly records")


📊 Step 8: Merge all features to monthly frequency...
   ✅ Daily data aggregation completed, total 720,831  monthly records


In [ ]:
# Merge Market capitalization and bm
# Note: need to unify date format, use year and month for merge

# Ensure _monthly has year and month columns
if 'year' not in _monthly.columns or 'month' not in _monthly.columns:
    print("   ⚠️  _monthly missing year or month columns, adding...")
    _monthly['year'] = _monthly['date'].dt.year
    _monthly['month'] = _monthly['date'].dt.month

# Ensure crsp_monthly has year and month columns
if 'year' not in crsp_monthly.columns or 'month' not in crsp_monthly.columns:
    crsp_monthly['year'] = crsp_monthly['date'].dt.year
    crsp_monthly['month'] = crsp_monthly['date'].dt.month

# Check if already merged (avoid duplicate merge)
if 'mve' not in _monthly.columns or 'bm' not in _monthly.columns:
    print("   - Merge Market capitalization and bm data...")
    _monthly = _monthly.merge(
        crsp_monthly[['permno', 'year', 'month', 'mve', 'bm']],
        on=['permno', 'year', 'month'],
        how='left'
    )
    print(f"   - After merging Market capitalization data, mve valid values: {_monthly['mve'].notna().sum():,}")
else:
    print("   - mve and bm already exist, skip merge")

# Merge beta and idiovol (aggregated from daily data)
if 'beta' not in _monthly.columns or 'idiovol' not in _monthly.columns:
    print("   - Merge beta and idiovol data...")
    crsp_beta['year'] = crsp_beta['date'].dt.year
    crsp_beta['month'] = crsp_beta['date'].dt.month
    beta_monthly = crsp_beta.groupby(['permno', 'year', 'month']).last().reset_index()
    _monthly = _monthly.merge(
        beta_monthly[['permno', 'year', 'month', 'beta', 'betasq', 'idiovol']],
        on=['permno', 'year', 'month'],
        how='left'
    )
    print(f"   - After merging beta data, beta valid values: {_monthly['beta'].notna().sum():,}")
else:
    print("   - beta and idiovol already exist, skip merge")

# Merge Profitability features (match using year)
if 'operprof' not in _monthly.columns or 'roeq' not in _monthly.columns:
    if 'compustat_prof_dec' in locals() and len(compustat_prof_dec) > 0:
        print(" - Merge Profitability data...")
        print(f" - compustat_prof_dec columns: {list(compustat_prof_dec.columns)}")
        
        # Check and delete duplicate columns (including those with _x, _y suffixes)
        cols_to_drop = []
        for col in ['operprof', 'roeq']:
            # Check original column names
            if col in _monthly.columns:
                print(f" ⚠️  Found duplicate column {col}，Delete old column first")
                cols_to_drop.append(col)
            # Check columns with suffixes
            for suffix in ['_x', '_y', '_new']:
                suffixed_col = f'{col}{suffix}'
                if suffixed_col in _monthly.columns:
                    print(f" ⚠️  Found duplicate column {suffixed_col}，delete first")
                    cols_to_drop.append(suffixed_col)
        
        if cols_to_drop:
            _monthly = _monthly.drop(columns=cols_to_drop)
        
        # Merge data
        _monthly = _monthly.merge(
            compustat_prof_dec,
            on=['permno', 'year'],
            how='left'
        )
        
        # Safely check if column exists
        if 'operprof' in _monthly.columns:
            print(f" - After merging Profitability data, operprof valid values: {_monthly['operprof'].notna().sum():,}")
        else:
            print(" ⚠️  operprof column does not exist after merge")
        if 'roeq' in _monthly.columns:
            print(f" - After merging Profitability data, roeq valid values: {_monthly['roeq'].notna().sum():,}")
        else:
            print(" ⚠️  roeq column does not exist after merge")
    else:
        print(" ⚠️  compustat_prof_dec does not exist or is empty, skip Profitability data merge")
else:
    print(" - Profitability data already exists, skip merge")

# Merge cashpr (cash price ratio, match using year)
if 'cashpr' not in _monthly.columns:
    if 'compustat_cash' in locals() and len(compustat_cash) > 0:
        print("   - Merge cashpr data...")
        compustat_cash_year = compustat_cash[['permno', 'year', 'cashpr']].copy()
        _monthly = _monthly.merge(
            compustat_cash_year,
            on=['permno', 'year'],
            how='left'
        )
        print(f" - After merging cashpr data, cashpr valid values: {_monthly['cashpr'].notna().sum():,}")
    else:
        print(" - compustat_cash does not exist, skip cashpr merge")
else:
    print(" - cashpr already exists, skip merge")

# Merge Industry codes (need to unify date format)
# First clean duplicate sic2 columns (if there are _x, _y suffixes)
sic2_cols = [col for col in _monthly.columns if 'sic2' in col]
if len(sic2_cols) > 1:
    print(f" - Found duplicate sic2 columns: {sic2_cols}，cleaning...")
    # Keep the first non-empty sic2 column
    for col in sic2_cols:
        if col != 'sic2' and _monthly[col].notna().sum() > 0:
            if 'sic2' not in _monthly.columns:
                _monthly['sic2'] = _monthly[col]
            else:
                _monthly['sic2'] = _monthly['sic2'].fillna(_monthly[col])
    # Delete all columns with suffixes
    _monthly = _monthly.drop(columns=[col for col in sic2_cols if col != 'sic2'])

if 'sic2' not in _monthly.columns or _monthly['sic2'].isna().all():
    if len(crsp_names) > 0:
        print(" - Merge Industry code data...")
        # Ensure crsp_names has sic2 column
        if 'sic2' not in crsp_names.columns:
            if 'siccd' in crsp_names.columns:
                crsp_names['sic2'] = crsp_names['siccd'].astype(str).str[:2]
            else:
                print(" ⚠️  crsp_names does not have siccd or sic2 column")
                crsp_names['sic2'] = None
        
        crsp_names['year'] = crsp_names['date'].dt.year
        crsp_names['month'] = crsp_names['date'].dt.month
        
        # For each permno-year-month, take the latest sic2 (if there are multiple)
        crsp_names_monthly = crsp_names.groupby(['permno', 'year', 'month'])['sic2'].last().reset_index()
        
        # If _monthly already has sic2 column, delete first (avoid duplicate)
        if 'sic2' in _monthly.columns:
            _monthly = _monthly.drop(columns=['sic2'])
        
        _monthly = _monthly.merge(
            crsp_names_monthly,
            on=['permno', 'year', 'month'],
            how='left'
        )
        print(f"- After merging Industry data, sic2 valid values: {_monthly['sic2'].notna().sum():,}")
    else:
        print(" ⚠️  crsp_names is empty, skip Industry data merge")
        if 'sic2' not in _monthly.columns:
            _monthly['sic2'] = None
else:
    print(" - sic2 already exists and has data, skip merge")

# Clean duplicate columns (handle _x, _y suffixes)
print(" - Clean duplicate columns...")
cols_to_drop = []
cols_to_rename = {}

for col in _monthly.columns:
    if col.endswith('_x') or col.endswith('_y'):
        base_col = col[:-2]
        if base_col in _monthly.columns:
            # If base column exists, delete columns with suffixes
            cols_to_drop.append(col)
        else:
            # If base column does not exist, rename to remove suffix
            cols_to_rename[col] = base_col

# Rename first
if cols_to_rename:
    _monthly = _monthly.rename(columns=cols_to_rename)
    print(f" - Renamed {len(cols_to_rename)}   columns (remove suffix)")

# Only delete columns that actually exist
cols_to_drop = [col for col in cols_to_drop if col in _monthly.columns]
if cols_to_drop:
    _monthly = _monthly.drop(columns=cols_to_drop)
    print(f" - Deleted {len(cols_to_drop)}   duplicate columns")
else:
    print(" - No duplicate columns to delete")

print(" ✅ All features merged")


   - Merge market capitalization and bm data...
   - After merging market capitalization data, mve valid values: 716,975
   - Merge beta and idiovol data...
   - After merging beta data, beta valid values: 564,101
   - Merge profitability data...
   - compustat_prof_dec columns: ['permno', 'year', 'operprof', 'roeq']
   ⚠️  Found duplicate column roeq，Delete old column first
   - After merging profitability data, operprof valid values: 237,279
   - After merging profitability data, roeq valid values: 341,373
   - cashpr already exists, skip merge
   - sic2 already exists and has data, skip merge
   - Clean duplicate columns...
   - Renamed 2  columns (removed suffix)
   - No duplicate columns to delete
   ✅ All features merged


In [ ]:
# Rename to match Project Variable Name
# First check and clean duplicate columns, then rename

print("   - Check current column names...")
print(f"   - Current columns: {list(_monthly.columns)}")

# Define rename mapping
rename_map = {
    'mom1m': 'ret_1_0',
    'mom6m': 'ret_6_1',
    'mom12m': 'ret_12_1',
    'mom36m': 'ret_36_1',
    'maxret': 'rmax1_21d',
    'retvol': 'rvol_21d',
    'turn_126d': 'turnover_126d',
    'dolvol_126d': 'dolvol_126d',
    'zerotrade': 'zero_trades_252d',
    'baspread_21d': 'bidaskhl_21d',
    'mve': 'me',
    'bm': 'be_me',
    'beta': 'beta_60m',
    'idiovol': 'ivol_capm_252d',
    'operprof': 'qmj_prof'
}

# Only rename existing columns
rename_map_filtered = {k: v for k, v in rename_map.items() if k in _monthly.columns}

# Check if target column names already exist (may be duplicate columns)
for old_col, new_col in rename_map_filtered.items():
    if new_col in _monthly.columns and old_col in _monthly.columns:
        # If both old and new columns exist, merge data (fill old column nulls with new column)
        if old_col != new_col:
            print(f" ⚠️  {new_col} already exists, merge {old_col}  data...")
            _monthly[new_col] = _monthly[new_col].fillna(_monthly[old_col])
            # Delete old column
            _monthly = _monthly.drop(columns=[old_col])
            # Remove from rename mapping
            rename_map_filtered.pop(old_col)

# Execute rename
if rename_map_filtered:
    _monthly = _monthly.rename(columns=rename_map_filtered)
    print(f" - Renamed {len(rename_map_filtered)}   columns")

# Final cleanup: delete all duplicate columns (if any)
print(" - Final cleanup of duplicate columns...")

# Check if there are duplicate column names
if len(_monthly.columns) != len(set(_monthly.columns)):
    print(" ⚠️  Found duplicate column names, cleaning...")
    # Find duplicate column names and their positions
    seen = {}
    duplicate_info = {}
    for i, col in enumerate(_monthly.columns):
        if col in seen:
            if col not in duplicate_info:
                duplicate_info[col] = [seen[col], i]
            else:
                duplicate_info[col].append(i)
        else:
            seen[col] = i
    
    # Merge duplicate column data (fill first column nulls with subsequent columns)
    for col_name, indices in duplicate_info.items():
        first_idx = indices[0]
        # Directly use iloc to access by position, avoid column name duplication issues
        first_col_data = _monthly.iloc[:, first_idx].copy()
        
        for dup_idx in indices[1:]:
            dup_col_data = _monthly.iloc[:, dup_idx].copy()
            # Fill first column nulls with duplicate column data
            mask = first_col_data.isna()
            first_col_data.loc[mask] = dup_col_data.loc[mask]
        
        # Update first column data
        _monthly.iloc[:, first_idx] = first_col_data
    
    # Delete duplicate columns (keep first)
    # Use column names to deduplicate, but need to delete by position
    cols_to_keep = []
    seen_cols = set()
    for i, col in enumerate(_monthly.columns):
        if col not in seen_cols:
            cols_to_keep.append(i)
            seen_cols.add(col)
    
    _monthly = _monthly.iloc[:, cols_to_keep]
    print(f" - Cleaned {sum(len(indices)-1 for indices in duplicate_info.values())}   duplicate columns")
else:
    print(" - No duplicate columns")

print(" ✅ Variable renaming completed")
print(f"\n📊 Final data frame shape: {_monthly.shape}")
print(f" Column names: {list(_monthly.columns)}")
print(f" Unique column count: {len(set(_monthly.columns))}")

# Safely check valid values
if 'me' in _monthly.columns:
    me_count = int(_monthly['me'].notna().sum())
    print(f" - me valid values: {me_count:,}")
else:
    print(" - me column does not exist")

if 'be_me' in _monthly.columns:
    be_me_count = int(_monthly['be_me'].notna().sum())
    print(f" - be_me valid values: {be_me_count:,}")
else:
    print(" - be_me column does not exist")

_monthly.head()


   - Check current column names...
   - Current columns: ['permno', 'date', 'ret_1_0', 'ret_6_1', 'ret_12_1', 'ret_36_1', 'chmom', 'rmax1_21d', 'rvol_21d', 'turnover_126d', 'std_turn', 'dolvol_126d', 'zero_trades_252d', 'bidaskhl_21d', 'me', 'me', 'me', 'be_me', 'be_me', 'be_me', 'beta_60m', 'beta_60m', 'beta_60m', 'betasq', 'betasq', 'ivol_capm_252d', 'ivol_capm_252d', 'ivol_capm_252d', 'year', 'qmj_prof', 'qmj_prof', 'qmj_prof', 'month', 'cashpr', 'sic2', 'roeq']
   - Renamed 1  columns
   - Final cleanup of duplicate columns...
   ⚠️  Found duplicate column names, cleaning...
   - Cleaned 11  duplicate columns
   ✅ Variable renaming completed

📊 Final data frame shape: (720831, 25)
   Column names: ['permno', 'date', 'ret_1_0', 'ret_6_1', 'ret_12_1', 'ret_36_1', 'chmom', 'rmax1_21d', 'rvol_21d', 'turnover_126d', 'std_turn', 'dolvol_126d', 'zero_trades_252d', 'bidaskhl_21d', 'me', 'be_me', 'beta_60m', 'betasq', 'ivol_capm_252d', 'year', 'qmj_prof', 'month', 'cashpr', 'sic2', 'roeq']


,permno,date,ret_1_0,ret_6_1,ret_12_1,ret_36_1,chmom,rmax1_21d,rvol_21d,turnover_126d,...,be_me,beta_60m,betasq,ivol_capm_252d,year,qmj_prof,month,cashpr,sic2,roeq
0,10026,2018-01-01,NaN,NaN,NaN,NaN,NaN,0.036752,0.021137,NaN,...,<NA>,NaN,NaN,NaN,2018,<NA>,1,<NA>,20,<NA>
1,10026,2018-02-01,-0.084515,NaN,NaN,NaN,NaN,0.027858,0.020436,NaN,...,<NA>,NaN,NaN,NaN,2018,<NA>,2,<NA>,20,<NA>
2,10026,2018-03-01,-0.008108,NaN,NaN,NaN,NaN,0.024013,0.012014,4.257142,...,<NA>,NaN,NaN,NaN,2018,<NA>,3,<NA>,20,<NA>
3,10026,2018-04-01,0.033132,NaN,NaN,NaN,NaN,0.023609,0.011150,3.982856,...,<NA>,NaN,NaN,NaN,2018,<NA>,4,<NA>,20,<NA>
4,10026,2018-05-01,0.045195,NaN,NaN,NaN,NaN,0.030120,0.013237,3.957697,...,<NA>,NaN,NaN,NaN,2018,<NA>,5,<NA>,20,<NA>


## 11. Data quality check

Check data integrity, missing values, etc.


In [ ]:
# Data quality check
print("📊 Data quality check:")
print(f" Total records: {len(_monthly):,}")
print(f" Number of stocks: {_monthly['permno'].nunique():,}")
print(f" Date range: {_monthly['date'].min()} to {_monthly['date'].max()}")

# Check missing values
print("\n📊 Missing value statistics:")
missing_stats = _monthly.isnull().sum()
missing_pct = (missing_stats / len(_monthly) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing count': missing_stats,
    'Missing percentage (%)': missing_pct
})
missing_df = missing_df[missing_df['Missing count'] > 0].sort_values('Missing count', ascending=False)
print(missing_df)

# Display feature summary
feature_cols = [c for c in _monthly.columns if c not in ['permno', 'date', 'year', 'month']]
print(f"\n📊 Feature summary:")
print(f" Total number of : {len(feature_cols)}")
print(f" Feature list: {feature_cols}")


📊 Data quality check:
   Total records: 720,831
   Number of stocks: 13,854
   Date range: 2018-01-01 00:00:00 to 2024-12-01 00:00:00

📊 Missing value statistics:
                    Missing count  Missing percentage (%)
qmj_prof          483552    67.08
cashpr            451944    62.70
ret_36_1          407173    56.49
roeq              379458    52.64
be_me             366190    50.80
ret_12_1          156730    21.74
chmom             156730    21.74
ivol_capm_252d    156730    21.74
betasq            156730    21.74
beta_60m          156730    21.74
ret_6_1            99735    13.84
zero_trades_252d   71845     9.97
dolvol_126d        32653     4.53
std_turn           32653     4.53
turnover_126d      32653     4.53
ret_1_0            13836     1.92
me                  3856     0.53
bidaskhl_21d        2747     0.38
rvol_21d            2747     0.38
rmax1_21d           2747     0.38

📊 Feature summary:
   Total number of : 21
   Feature list: ['ret_1_0', 'ret_6_1', 'ret_12_1', 're

## 12. Save data

Save the extracted  as a CSV file.


In [ ]:
print(f"💾 Saving data to {OUTPUT_FILE}...")
_monthly.to_csv(OUTPUT_FILE, index=False)
print(f" ✅ Data saved！")
print(f" - File: {OUTPUT_FILE}")
print(f" - Number of rows: {len(_monthly):,}")
print(f" - Number of columns: {len(_monthly.columns)}")

print("\n✅ All feature extraction completed！")
print("\n📋 List of extracted :")
feature_cols = [c for c in _monthly.columns if c not in ['permno', 'date', 'year', 'month']]
for i, col in enumerate(feature_cols, 1):
    print(f"   {i:2d}. {col}")


💾 Saving data to wrds_20_.csv...
   ✅ Data saved！
   - File: wrds_20_.csv
   - Number of rows: 720,831
   - Number of columns: 25

✅ All feature extraction completed！

📋 List of extracted :
    1. ret_1_0
    2. ret_6_1
    3. ret_12_1
    4. ret_36_1
    5. chmom
    6. rmax1_21d
    7. rvol_21d
    8. turnover_126d
    9. std_turn
   10. dolvol_126d
   11. zero_trades_252d
   12. bidaskhl_21d
   13. me
   14. be_me
   15. beta_60m
   16. betasq
   17. ivol_capm_252d
   18. qmj_prof
   19. cashpr
   20. sic2
   21. roeq
